In [33]:
from DataModelDict import DataModelDict as DM
from pathlib import Path

import uuid

import potentials

import requests

In [2]:
machine = 'laptop'
#machine = 'desktop'

In [3]:
# Set passwordfile based on machine
if machine == 'laptop':
    passwordfile = Path('C:/Users/lmh1/Documents/potentials_nist_gov/password.txt')
elif machine == 'desktop':
    passwordfile = Path('E:/potentials_nist_gov/password.txt')
else:
    raise ValueError(f'passwordfile not set for machine {machine}')

# Load database using username + password
with open(Path(passwordfile)) as f:
    username, password = f.read().strip().split()
db = potentials.Database(username=username, password=password)

In [32]:
db.load_potentials(remote=False, verbose=True)
db.set_installed_kim_models([])
db.load_lammps_potentials(remote=False, verbose=True)

Loaded 640 local potentials
Loaded 397 local LAMMPS potentials
Building lammps potentials for kim models
No installed kim models identified


In [84]:
def get_fullkimids():
    """Scrapes the openKIM website for the list of all current KIM models"""
    r = requests.get('https://openkim.org/browse/models/alphabetical')
    fullids = []
    s = 0
    e = 0
    while True:
        try:
            s = r.text[e:].index('<a href="/id/') + 13 + e
            e = r.text[s:].index('">') + s
            fullids.append(r.text[s:e])            
        except:
            break
    
    return fullids

def previouskimid(fullkimid):
    """Scrapes the openKIM website to find the previous version of a kim model"""
    r = requests.get(f'https://openkim.org/id/{fullkimid}')
    s = r.text.index('<span class=title>Previous Version</span>')
    s = r.text[s:].index('<a href="/id/') + s + 13
    e = r.text[s:].index('">') + s
    return r.text[s:e]

def getshortcode(fullkimid):
    """Extracts the kim shortcode from the fullkimid"""
    return '_'.join(fullkimid.split('_')[-3:-1])

def sortfxn(kimid):
    return int(kimid.split('_')[-1])

def build_kim_record(potdb, fullkimid):
    
    shortcode = getshortcode(fullkimid)
    comment = ''
    
    # Load existing record
    fname = Path(potdb.localpath, 'potential_LAMMPS_KIM', f'{shortcode}.json')
    if fname.is_file():
        model = DM(fname)

    # Initialize new record
    else:
        comment += ' - is new\n'
        model = DM()
        model['potential-LAMMPS-KIM'] = DM()
        model['potential-LAMMPS-KIM']['key'] = None
        model['potential-LAMMPS-KIM']['id'] = shortcode

    # Add associated potential information if missing
    comment += addpotinfo(potdb, model)        

    # Get previous versions
    allids = [fullkimid]
    version = fullkimid.split('_')[-1]
    while version != '000':
        allids.append(previouskimid(allids[-1]))
        version = allids[-1].split('_')[-1]
    allids = sorted(allids, key=sortfxn)

    saveids = model['potential-LAMMPS-KIM'].aslist('full-kim-id')
    count = 0
    for fid in allids:
        if fid not in saveids:
            model['potential-LAMMPS-KIM'].append('full-kim-id', fid)
            count += 1
    if count > 0:
        comment += f' - {count} versions added\n'

    # Reorder
    model['potential-LAMMPS-KIM']['key'] = model['potential-LAMMPS-KIM'].pop('key')
    model['potential-LAMMPS-KIM']['id'] = model['potential-LAMMPS-KIM'].pop('id')
    if 'potential' in model['potential-LAMMPS-KIM']:
        model['potential-LAMMPS-KIM']['potential'] = model['potential-LAMMPS-KIM'].pop('potential')
    model['potential-LAMMPS-KIM']['full-kim-id'] = model['potential-LAMMPS-KIM'].pop('full-kim-id')

    if comment != '':
        print(shortcode)
        print(comment)
    
    #with open(fname, 'w') as f:
    #    model.json(fp=f, indent=4)

def addpotinfo(potdb, model):
    
    shortcode = model['potential-LAMMPS-KIM']['id']
    comment = ''
    
    for potential in potdb.potentials:
        match = False
        for implementation in potential.implementations:
            if implementation.id == shortcode:
                match = True
                break
        
        if match:
            
            # Set/check key
            if model['potential-LAMMPS-KIM']['key'] is None:
                model['potential-LAMMPS-KIM']['key'] = implementation.key
            elif model['potential-LAMMPS-KIM']['key'] != implementation.key:
                comment += f' - key mismatch with {potential.id} {implementation.id}\n' 
            
            # Search for existing potential listing
            newpot = True
            for potinfo in model['potential-LAMMPS-KIM'].aslist('potential'):
                if potinfo['key'] == potential.key:
                    newpot = False
                    break
            
            if newpot:            
                # Build potential info
                potinfo = DM()
                model['potential-LAMMPS-KIM'].append('potential', potinfo)
                potinfo['key'] = potential.key
                potinfo['id'] = potential.id
                comment += ' - new potential info added\n'
            
            # Add atom info
            if 'atom' not in potinfo:
                noatom = True
                for implementation in potential.implementations:
                    try:
                        sibling = potdb.get_lammps_potential(id = implementation.id)
                    except:
                        pass
                    else:
                        potinfo['atom'] = sibling['potential-LAMMPS']['atom']
                        noatom = False
                        break
            
                if noatom:
                    count = 0
                    for element in potential.elements:
                        potinfo.append('atom', DM([('element', element)]))
                        count += 0
                    if count > 0:
                        comment += f' - {count} new atoms added\n'
                    if 'atom' not in potinfo:
                        comment += f' - has no atom info for {potential.id}\n'
    
    # Check if potential is in model after function
    if 'potential' not in model['potential-LAMMPS-KIM']:
        comment += ' - has no matching potential\n'

    return comment

def update_kim_models(potdb):
    fullkimids = get_fullkimids()
    
    for fullkimid in fullkimids:
        
        # Skip non-native KIM models (for now)
        if '__MO_' not in fullkimid:
            continue
        
        build_kim_record(potdb, fullkimid)
    
    
    
    

In [85]:
update_kim_models(db)

MO_546673549085
 - has no matching potential

MO_848899341753
 - has no matching potential

MO_463728687265
 - has no matching potential

MO_028979335952
 - has no matching potential

MO_135739722270
 - has no matching potential

MO_958395190627
 - has no matching potential

MO_390178379548
 - has no matching potential

MO_492310898779
 - has no matching potential

MO_700541006254
 - has no matching potential

MO_427873955970
 - has no matching potential

MO_115454747503
 - has no matching potential

MO_410167849923
 - has no matching potential

MO_179158257180
 - has no matching potential

MO_808662295149
 - has no matching potential

MO_671124822359
 - has no matching potential

MO_348689608050
 - has no matching potential

MO_131642768288
 - has no matching potential

MO_557492625287
 - has no matching potential

MO_093637366498
 - is new
 - has no matching potential
 - 1 versions added

MO_099716416216
 - has no matching potential

MO_815057898706
 - has no matching potential

MO_8

In [42]:
model.finds('atom')

[DataModelDict([('element', 'Al')]),
 DataModelDict([('element', 'Ag')]),
 DataModelDict([('element', 'Au')]),
 DataModelDict([('element', 'Cu')]),
 DataModelDict([('element', 'Ni')]),
 DataModelDict([('element', 'Pd')]),
 DataModelDict([('element', 'Pt')])]

In [69]:
def addpotinfo(potdb, model):
    
    shortcode = model['potential-LAMMPS-KIM']['id']
    
    for potential in potdb.potentials:
        match = False
        for implementation in potential.implementations:
            if implementation.id == shortcode:
                match = True
                break
        
        if match:
            
            # Set/check key
            if model['potential-LAMMPS-KIM']['key'] is None:
                model['potential-LAMMPS-KIM']['key'] = implementation.key
            elif model['potential-LAMMPS-KIM']['key'] != implementation.key:
                print(shortcode, 'key mismatch with', potential.id, implementation.id)
            
            # Search for existing potential listing
            newpot = True
            for potinfo in model['potential-LAMMPS-KIM'].aslist('potential'):
                if potinfo['key'] == potential.key:
                    newpot = False
                    break
            
            if newpot:            
                # Build potential info
                potinfo = DM()
                model['potential-LAMMPS-KIM'].append('potential', potinfo)
                potinfo['key'] = potential.key
                potinfo['id'] = potential.id
            
            # Add atom info
            if 'atom' not in potinfo:
                noatom = True
                for implementation in potential.implementations:
                    try:
                        sibling = potdb.get_lammps_potential(id = implementation.id)
                    except:
                        pass
                    else:
                        potinfo['atom'] = sibling['potential-LAMMPS']['atom']
                        noatom = False
                        break
            
                if noatom:
                    for element in potential.elements:
                        potinfo.append('atom', DM([('element', element)]))
                    if 'atom' not in potinfo:
                        print(shortcode, 'has no atom info for', potential.id)
    
    # Check if potential is in model after function
    if 'potential' not in model['potential-LAMMPS-KIM']:
        print(shortcode, 'has no matching potential')


In [70]:
for p in db.potentials:
    print(p.elements)


['Ar']
['Kr']
['Ne']
['Xe']
['Ag']
['Al']
['Ba']
['Ca']
['Cr']
['Cs']
['Cu']
['Fe']
['K']
['Mo']
['Na']
['Ni']
['Pb']
['Rb']
['Sr']
['W']
['Ne']
['Ar']
['Ar', 'Ne']
['Cu']
['Ni', 'Cu']
['Si']
['Ag']
['Ag', 'Au', 'Cu', 'Ni', 'Pd', 'Pt']
['Au']
['Cu']
['Ni']
['Pd']
['Pt']
['Si']
['Mo']
['Nb']
['Ta']
['V']
['W']
['Ag']
['Au']
['Cu']
['Ni']
['Si']
['Al']
['Cu']
['Si']
['C']
['Ge']
['Si']
['C']
['Si']
['Si']
['Ag']
['Ag', 'Au', 'Cu', 'Ni', 'Pd', 'Pt']
['Au']
['Cu']
['Ni']
['Pd']
['Pt']
['Si']
['Si', 'C']
['Si', 'Ge']
['Cd', 'Te']
['Pt']
['Cu']
['Si', 'C']
['Si', 'O']
['Si']
['Ti']
['Si']
['Si']
['Si']
['Al']
['Si', 'O']
['Si', 'C']
['Zr']
['Ni']
['Ni', 'Al', 'H']
['Al', 'Cu']
['Nb', 'Ti', 'Al']
['Ag']
['Al']
['Al', 'Ag', 'Au', 'Cu', 'Ni', 'Pd', 'Pt']
['Au']
['Cu']
['Ni']
['Pd']
['Pt']
['Si']
['Fe']
['Si', 'O']
['Al', 'Mg']
['Co', 'Al']
['Si', 'C']
['Si']
['Al', 'Mg']
['Fe']
['Si']
['Al', 'Cu']
['Al']
['Ni']
['Al', 'Pb']
['Si']
['H', 'He', 'C', 'Ar', 'Xe']
['Al']
['Cr']
['Fe']
['Mo']
['Nb']


In [71]:
model = DM()

In [72]:
model = DM('C:/Users/lmh1/Documents/library/potential_LAMMPS_KIM/MO_115316750986.json')
model['potential-LAMMPS-KIM']['key'] = '235'
#del model['potential-LAMMPS-KIM']['potential']
print(model.json(indent=4))

{
    "potential-LAMMPS-KIM": {
        "key": "235",
        "id": "MO_115316750986",
        "potential": {
            "key": "dd581609-b17f-433a-952a-e746a2bf1d4f",
            "id": "1996--Jacobsen-K-W-Stoltze-P-Norskov-J-K--Al-Ag-Au-Cu-Ni-Pd-Pt",
            "atom": [
                {
                    "element": "Al"
                },
                {
                    "element": "Ag"
                },
                {
                    "element": "Au"
                },
                {
                    "element": "Cu"
                },
                {
                    "element": "Ni"
                },
                {
                    "element": "Pd"
                },
                {
                    "element": "Pt"
                }
            ]
        },
        "full-kim-id": [
            "EMT_Asap_Standard_JacobsenStoltzeNorskov_1996_AlAgAuCuNiPdPt__MO_115316750986_000",
            "EMT_Asap_Standard_JacobsenStoltzeNorskov_1996_AlAgAuCuN

In [73]:
addpotinfo(db, model)
print(model.json(indent=4))

MO_115316750986 key mismatch with 1996--Jacobsen-K-W-Stoltze-P-Norskov-J-K--Al-Ag-Au-Cu-Ni-Pd-Pt MO_115316750986
{
    "potential-LAMMPS-KIM": {
        "key": "235",
        "id": "MO_115316750986",
        "potential": {
            "key": "dd581609-b17f-433a-952a-e746a2bf1d4f",
            "id": "1996--Jacobsen-K-W-Stoltze-P-Norskov-J-K--Al-Ag-Au-Cu-Ni-Pd-Pt",
            "atom": [
                {
                    "element": "Al"
                },
                {
                    "element": "Ag"
                },
                {
                    "element": "Au"
                },
                {
                    "element": "Cu"
                },
                {
                    "element": "Ni"
                },
                {
                    "element": "Pd"
                },
                {
                    "element": "Pt"
                }
            ]
        },
        "full-kim-id": [
            "EMT_Asap_Standard_JacobsenStoltzeNorsko

In [31]:
scrape_kim_models()

MO_800536961967
['EAM_CubicNaturalSpline_Angelo_Moody_Ni__MO_800536961967_000', 'EAM_CubicNaturalSpline_AngeloMoody_1995_Ni__MO_800536961967_001', 'EAM_CubicNaturalSpline_AngeloMoodyBaskes_1995_Ni__MO_800536961967_002', 'EAM_CubicNaturalSpline_AngeloMoodyBaskes_1995_Ni__MO_800536961967_003']


In [ ]:
{
    "potential-LAMMPS-KIM": {
        "key": "d3694758-2a82-4f0b-933c-d66a7a6aa6a2",
        "id": "MO_004835508849",
        "potential": {
            "key": "bd58ae4c-dc17-40e4-91f0-db3eae46f933",
            "id": "2007--Mendelev-M-I-Ackland-G-J--Zr-3",
            "atom": {
                "element": "Zr",
                "mass": 91.224
            }
        },
        "full-kim-id": "EAM_Dynamo_MendelevAckland_2007v3_Zr__MO_004835508849_000"
    }
}

In [22]:
shortcode('EAM_Dynamo_HaleWongZimmerman_2008PairHybrid_PdAgH__MO_104806802344_005')

'MO_104806802344'

In [18]:
fullids

['DUNN_WenTadmor_2019v1_C__MO_584345505904_000',
 'DUNN_WenTadmor_2019v2_C__MO_956135237832_000',
 'DUNN_WenTadmor_2019v3_C__MO_714772088128_000',
 'EAM_CubicNaturalSpline_AngeloMoodyBaskes_1995_Ni__MO_800536961967_003',
 'EAM_CubicNaturalSpline_ErcolessiAdams_1994_Al__MO_800509458712_002',
 'EAM_Dynamo_Ackland_1987_Au__MO_754413982908_000',
 'EAM_Dynamo_Ackland_1992_Ti__MO_748534961139_005',
 'EAM_Dynamo_Ackland_2003_W__MO_141627196590_005',
 'EAM_Dynamo_AcklandBaconCalder_1997_Fe__MO_142799717516_005',
 'EAM_Dynamo_AcklandMendelevSrolovitz_2004_FeP__MO_884343146310_005',
 'EAM_Dynamo_AcklandTichyVitek_1987_Ag__MO_212700056563_005',
 'EAM_Dynamo_AcklandTichyVitek_1987_Au__MO_104891429740_005',
 'EAM_Dynamo_AcklandTichyVitek_1987_Cu__MO_179025990738_005',
 'EAM_Dynamo_AcklandTichyVitek_1987_Ni__MO_977363131043_005',
 'EAM_Dynamo_AcklandTichyVitek_1987v2_Ag__MO_055919219575_000',
 'EAM_Dynamo_AcklandTichyVitek_1987v2_Cu__MO_762798677854_000',
 'EAM_Dynamo_AcklandTichyVitek_1987v2_Ni__MO

In [14]:
r.text.index('not here')

ValueError: substring not found

In [5]:
records = {}
for filename in Path(db.localpath, 'potential_LAMMPS_KIM').glob('MO_*'):
    if filename.suffix.lower() not in ['.json', '.xml']:
        continue
    recordname = filename.stem
    records[recordname] = filename
  
print(len(records), 'local potential_LAMMPS_KIM records found')

450 local potential_LAMMPS_KIM records found


In [6]:
records

{'MO_004835508849': WindowsPath('C:/Users/lmh1/Documents/library/potential_LAMMPS_KIM/MO_004835508849.json'),
 'MO_008996216289': WindowsPath('C:/Users/lmh1/Documents/library/potential_LAMMPS_KIM/MO_008996216289.json'),
 'MO_010059867259': WindowsPath('C:/Users/lmh1/Documents/library/potential_LAMMPS_KIM/MO_010059867259.json'),
 'MO_010613863288': WindowsPath('C:/Users/lmh1/Documents/library/potential_LAMMPS_KIM/MO_010613863288.json'),
 'MO_014123846623': WindowsPath('C:/Users/lmh1/Documents/library/potential_LAMMPS_KIM/MO_014123846623.json'),
 'MO_017524376569': WindowsPath('C:/Users/lmh1/Documents/library/potential_LAMMPS_KIM/MO_017524376569.json'),
 'MO_018428823000': WindowsPath('C:/Users/lmh1/Documents/library/potential_LAMMPS_KIM/MO_018428823000.json'),
 'MO_019616213550': WindowsPath('C:/Users/lmh1/Documents/library/potential_LAMMPS_KIM/MO_019616213550.json'),
 'MO_019873715786': WindowsPath('C:/Users/lmh1/Documents/library/potential_LAMMPS_KIM/MO_019873715786.json'),
 'MO_02084

In [9]:
template = 'potential_LAMMPS_KIM'
for title in records:
    content = DM(records[title]).xml()

    db.upload_record(template, content, title, verbose=True)

record MO_004835508849 (605b65a326ed1e002a28ef03) successfully uploaded.
record MO_008996216289 (605b65a726ed1e003828eef9) successfully uploaded.
record MO_010059867259 (605b65aa26ed1e003928eeef) successfully uploaded.
record MO_010613863288 (605b65ad26ed1e003b28eef4) successfully uploaded.
record MO_014123846623 (605b65b226ed1e003a28eeee) successfully uploaded.
record MO_017524376569 (605b65b526ed1e002a28ef07) successfully uploaded.
record MO_018428823000 (605b65b926ed1e003828eefd) successfully uploaded.
record MO_019616213550 (605b65bc26ed1e003928eef3) successfully uploaded.
record MO_019873715786 (605b65be26ed1e003b28eef8) successfully uploaded.
record MO_020840179467 (605b65c226ed1e003a28eef2) successfully uploaded.
record MO_020851069572 (605b65c826ed1e002a28ef0b) successfully uploaded.
record MO_022920256108 (605b65cb26ed1e003828ef01) successfully uploaded.
record MO_028979335952 (605b65ce26ed1e003928eef7) successfully uploaded.
record MO_034823476734 (605b65d226ed1e003b28eefc) s